# File to do Porto Taxi Trajectory Similiarity

### Step 1

Open the code/global_variables.py file, [or just click here](global_variables.py). And edit the values to fit the given experiment, the name of the chosen subset ("subset-*size*) and the size of the subset. As well as the coordinates of the geographical area.


### Step 2

Make sure you have the needed files/folders for the chosen subset.
 - in data/raw_data there must be a .csv file with the subset of the chosen size. If not, it must be uploaded.
 - in data/chosen_data there must be a folder with the same name as global_variables.CHOSEN_SUBSET_NAME. If not, create this empty folder.
 - in data/hashed_data there must be a folder with the same name as global_variables.CHOSEN_SUBSET_NAME. If not, create this empty folder.

### Step 3

Run the cells in [code/porto-data.ipynb](porto-data.ipynb), or just run the cell below.
This will load the data from the chosen subset into the folder data/chosen_data/subset-'size', each row in the dataset is written in its own text file. I also creates a META-file which contains the name of all the text files in the subset.

(Might requires to install nbformat: "pip install nbformat")

In [1]:
%run "porto-data.ipynb"

Check the folder: data/chosen_data/subset-subset-100. Files should have been generated.


### Step 4

Run the cells in [code/lsh-grid.ipynb](lsh-grid.ipynb), or just run the cell below. This will represent each of the rows/trajectories as an hash, and create a text file for each hashed trajectory in the folder data/hashed_data/subset-'size', as well as a META file.

In [2]:
%run "lsh-grid.ipynb"

       Average runtime  Minimum runtime  Maximum runtime
porto         0.021123         0.021031         0.021241
                 Average runtime  Minimum runtime  Maximum runtime
porto_naive             4.180326         4.131812          4.22105
porto_quadrants         2.111735         1.977048          2.29845
porto_kd_tree           2.121689         2.065953          2.19822
Check the folder: data/hashed_data/subset-100. Files should have been generated.


### Step 5
Calculate similarities by running the following code

In [3]:
%run similarities-only-grid.ipynb

Check ../code/experiments/similarities/, it should be a file here named grid_porto-subset-100.csv which contains the similarities in the dataset.
 Check ../code/experiments/timing/, it should be a file here named similarity_runtimes_grid_porto-{global_variables.CHOSEN_SUBSET_NAME}.csv which contains the time spent to do the hash similarity(?)


### Step 6

Run the code below to see the clustering of the trajectories. Decide the number of clusters you want by updating number_of_trajectories.

In [4]:
#Change this to the number of clusters you want (if wanted number is more than 30: update in def plot_clusters() in hierarchical_clustering.py)
number_of_clusters = 10

In [5]:
from experiments.hierarchical_clustering import HCA
from experiments import davies_bouldin as DB 
from sklearn import metrics as mcs

# Porto Grid similarities
#TODO: remove city
PortoGrid = HCA("Porto", f"../code/experiments/similarities/grid_porto-{global_variables.CHOSEN_SUBSET_NAME}.csv", number_of_clusters )
print(PortoGrid.clusters)
#PortoGrid.plot_clusters("Porto - Grid")
clusters_dict = PortoGrid.get_cluster_dictionary()
print(clusters_dict)

[0 4 5 5 0 1 6 8 2 8 2 3 0 2 2 2 9 2 0 5 2 1 5 0 0 2 3 8 0 3 4 5 6 0 2 5 8
 4 5 0 1 5 1 5 3 2 9 2 7 0 0 0 1 3 1 0 0 4 2 0 0 0 0 4 4 0 5 1 0 1 8 8 0 0
 0 9 0 4 2 0 0 4 0 0 2 0 8 0 9 2 3 0 5 0 0 4 5 4 0 4]
TEST123
['P_AAAA', 'P_AAAB', 'P_AAAC', 'P_AAAD', 'P_AAAE', 'P_AAAF', 'P_AAAG', 'P_AAAH', 'P_AAAI', 'P_AAAJ', 'P_AAAK', 'P_AAAL', 'P_AAAM', 'P_AAAN', 'P_AAAO', 'P_AAAP', 'P_AAAQ', 'P_AAAR', 'P_AAAS', 'P_AAAT', 'P_AAAU', 'P_AAAV', 'P_AAAW', 'P_AAAX', 'P_AAAY', 'P_AAAZ', 'P_AABA', 'P_AABB', 'P_AABC', 'P_AABD', 'P_AABE', 'P_AABF', 'P_AABG', 'P_AABH', 'P_AABI', 'P_AABJ', 'P_AABK', 'P_AABL', 'P_AABM', 'P_AABN', 'P_AABO', 'P_AABP', 'P_AABQ', 'P_AABR', 'P_AABS', 'P_AABT', 'P_AABU', 'P_AABV', 'P_AABW', 'P_AABX', 'P_AABY', 'P_AABZ', 'P_AACA', 'P_AACB', 'P_AACC', 'P_AACD', 'P_AACE', 'P_AACF', 'P_AACG', 'P_AACH', 'P_AACI', 'P_AACJ', 'P_AACK', 'P_AACL', 'P_AACM', 'P_AACN', 'P_AACO', 'P_AACP', 'P_AACQ', 'P_AACR', 'P_AACS', 'P_AACT', 'P_AACU', 'P_AACV', 'P_AACW', 'P_AACX', 'P_AACY', 'P_AACZ', 'P_AADA

/cluster/home/andrehva/.conda/envs/masteroppgave/lib/python3.12/site-packages/scipy/cluster/hierarchy.py:796: ClusterWarning: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  return linkage(y, method='ward', metric='euclidean')


In [7]:
from experiments.frechet_for_taxi_case import find_similarity_in_clusters
from experiments.frechet_for_taxi_case import find_similarity_in_cluster
from experiments.frechet_for_taxi_case import frechet_similar_taxi_trajectories
from experiments.frechet_for_taxi_case import frechet_similar_taxi_and_bus_trajectories
from experiments.frechet_for_taxi_case import merge_clusters


result = find_similarity_in_clusters(clusters_dict)
for r in result:
    print(len(r))
    print(r)

#liste = [['katt', 'hund'], ['hund', 'hest'], ['ku', 'sau'], ['geit', 'hest'], ['ku', 'okse']]
#liste2 = ['a', 'b', 'c', 'd', 'e', 'f']

#liste3 = liste2.copy()
#for e in liste2:
    #print(e)
    #if e in ['a', 'c', 'd']:
        #liste3.remove(e)
#print(liste3)
#result = merge_clusters(liste2)
#print(result)

102
['P_AAAE', 'P_AADE', 'P_AAAA', 'P_AAAY', 'P_AAAA', 'P_AABC', 'P_AAAA', 'P_AABZ', 'P_AAAA', 'P_AACD', 'P_AAAA', 'P_AADJ', 'P_AAAX', 'P_AABZ', 'P_AAAX', 'P_AADN', 'P_AAAY', 'P_AACW', 'P_AABC', 'P_AABZ', 'P_AABC', 'P_AADE', 'P_AABC', 'P_AADP', 'P_AABC', 'P_AADU', 'P_AABH', 'P_AADJ', 'P_AABH', 'P_AADQ', 'P_AABN', 'P_AADP', 'P_AABX', 'P_AADU', 'P_AABZ', 'P_AACW', 'P_AABZ', 'P_AADB', 'P_AACD', 'P_AACI', 'P_AACD', 'P_AACN', 'P_AACE', 'P_AADQ', 'P_AACH', 'P_AADQ', 'P_AACI', 'P_AACJ', 'P_AACJ', 'P_AADJ', 'P_AACK', 'P_AADJ', 'P_AACN', 'P_AACW', 'P_AACN', 'P_AADN', 'P_AACW', 'P_AADB', 'P_AADC', 'P_AADN', 'P_AADC', 'P_AADP', 'P_AADF', 'P_AADU', 'P_AADN', 'P_AADP', 'P_AAAE', 'P_AADN', 'P_AAAM', 'P_AACE', 'P_AAAX', 'P_AABY', 'P_AABH', 'P_AABY', 'P_AABH', 'P_AACK', 'P_AABH', 'P_AADF', 'P_AABN', 'P_AACU', 'P_AABN', 'P_AADF', 'P_AABX', 'P_AADB', 'P_AABY', 'P_AACI', 'P_AABY', 'P_AACQ', 'P_AABY', 'P_AADC', 'P_AACE', 'P_AACK', 'P_AACH', 'P_AADH', 'P_AACU', 'P_AADC', 'P_AACY', 'P_AADH', 'P_AADC', 'P_AA